<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Protodash Explainer

Protodash explainer takes as input a datapoint (or group of datapoints) that user wants to explain with respect to instances in a training set belonging to the same feature space. The method then tries to minimize the maximum mean discrepancy (MMD metric) between the input datapoints for which explanation is needed and a prespecified number of instances from the training set that it will select. In other words, it will try to select training instances that have the same distribution as the datapoints we want to explain. The method does greedy selection and has quality guarantees with it also returning importance weights for the chosen prototypical training instances indicative of how similar/representative they are.


The prototypical explanations in AIX360 are obtained using the Protodash algorithm developed in the following work: [ProtoDash: Fast Interpretable Prototype Selection](https://arxiv.org/abs/1707.01212)

In this tutorial we will see a examples of obtaining prototypes for users whose credit risk application was classified as Risk and No Risk . In each case, we showcase the top five prototypes from the training data along with how similar the feature values were for these prototypes.

## 1. Introduction <a name="introduction"></a>
The notebook will train a German Credit Risk model, and generate explanations using protodash explainer<br/>

### Contents
- [Introduction](#introduction)
- [Setup](#Setup)
- [Model building and evaluation](#model)
- [Configuration](#configuration)
- [Invokation via SDK](#sdk-invocation)
- [Explaination generation](#explanation-generation)

**Note:** This notebook should be run using with **Python 3.9.x** runtime. It requires service credentials for the following services:
  * Watson OpenScale <br/>

**Note** : **This notebook works only with Default python3.9 and Default Python 3.8 environments in case of WatsonStudio and CPD**

# Setup <a name="setup"></a>

### Package installation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade pyspark==3.3.1 | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1
!pip install numpy==1.23.5

### Action: restart the kernel!

### Configure credentials

In [ ]:
WOS_CREDENTIALS = {
    "url": "*****"
    "username": "***",
    "apikey": "****"
    "instance_id": "******"
}

## Model building and evaluation <a name="model"></a>
&ensp;&ensp;&ensp;In this section you will learn how to train sklearn model, run prediction and evaluate its output. 

### Load the training data

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

In [3]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
train_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


### Initialize spark session

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pandas as pd

sparkconf = SparkConf().setMaster("local[*]").set("spark.driver.extraJavaOptions", "-Xss512m").set("spark.executor.extraJavaOptions","-Xss512m")
spark = SparkSession.builder.appName("ProtodashExplainer").config(conf=sparkconf).getOrCreate()
spark.sparkContext._conf.getAll()

22/02/28 18:09:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 18:09:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/28 18:09:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


[('spark.driver.host', '192.168.1.110'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1646051949051'),
 ('spark.app.startTime', '1646051948345'),
 ('spark.executor.extraJavaOptions', '-Xss512m'),
 ('spark.driver.extraJavaOptions', '-Xss512m'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.port', '60677'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'ProtodashExplainer')]

In [5]:
trainingDF_spark = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
trainingDF_spark.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [6]:
spark_df = trainingDF_spark
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

Number of records for training: 4005
Number of records for evaluation: 995


In [7]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")

In [8]:
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [9]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
feature_filter = SQLTransformer(statement="SELECT * FROM __THIS__")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter, feature_filter])
model = pipeline.fit(train_data)

22/02/28 18:09:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.710353


# Set configuration <a name="configuration"></a>
User can set the following inputs for protodash explainer , if not set default values will be taken. 
* prototype_count : Number of prototypes to be detected (default : 5)
* explain_row_wise: Row wise generate or dataframe level prototype generation (default: False)

In [11]:
#Set protodash_inputs
protodash_inputs = {
                    "prototype_count": 5,
                    "explain_row_wise": True
                }
    

In [12]:
#Set metrics plugin level configuration
from ibm_metrics_plugin.common.utils.constants import ExplainabilityMetricType, MetricGroupType
configuration = {}
configuration['configuration'] = {
            "problem_type": "binary",
            "label_column": "Risk",
            "prediction": "predictedLabel",
            "input_data_type": "structured",
            "feature_columns": ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings","EmploymentDuration",
                "InstallmentPercent",
                "Sex",
                "OthersOnLoan",
                "CurrentResidenceDuration",
                "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"
            ],
            MetricGroupType.EXPLAINABILITY.value: {
                "metrics_configuration":{
                    ExplainabilityMetricType.PROTODASH.value: protodash_inputs
                }
            }
}

# Invokation via SDK  <a name="sdk-invokation"></a>

### Set up openscale client

In [13]:
from ibm_watson_openscale import APIClient as OpenScaleAPIClient

# For cloud
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator

wos_authenticator = IAMAuthenticator(
    apikey=WOS_CREDENTIALS["apikey"]
)

datamart_id = WOS_CREDENTIALS["instance_id"]

client = OpenScaleAPIClient(
    authenticator=wos_authenticator,
    service_instance_id=datamart_id
)
client.version

#For CPD
# from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

# authenticator = CloudPakForDataAuthenticator(
#     url=WOS_CREDENTIALS["url"],
#     username=WOS_CREDENTIALS["username"],
#     password=WOS_CREDENTIALS["password"],
#     disable_ssl_verification=True
# )

# client = OpenScaleAPIClient(
#     service_url=WOS_CREDENTIALS['url'],
#     authenticator=authenticator
# )

# client.version

'1.0.0'

# Explanation Generation <a name="explanation-generation"></a>

### Define the input df for which prototypes needs to be detected 
It is to ne noted that if the input data frame has single row , prototypes will be detected for single row .User has to adjust the protodash_inputs to generate prototypes per row (default) or for the complete input data

In [25]:
#Construct the test data df with predictions
test_df = predictions.toPandas()
print(test_df.iloc[199:201].shape[0])
testDF_spark = spark.createDataFrame(test_df.iloc[199:201])
#testDF_spark.printSchema()

2


### Run protodash explainer

In [16]:
results = client.ai_metrics.compute_metrics(spark=spark, configuration=configuration, data_frame=testDF_spark, training_data=trainingDF_spark)

Doing entitlement check on Cloud...Will skip as we don't have standard plan


{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:58 2022", "message_details": "Discovered groups : [<ibm_metrics_plugin.metrics.explainability.impl.explainability_metrics_evaluator.ExplainabilityMetricsEvaluator object at 0x141fd4370>]"}
{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:58 2022", "message_details": "explainability"}
{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:58 2022", "message_details": "Evaluating explainability metrics.."}
{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:58 2022", "message_details": "Evaluating Explainer: 'protodash'..."}
{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:58 2022", "message_details": "Evaluating protodash explainer"}
{"appname": "metrics-plugin", "loglevel": "INFO", "timestamp": "Mon Feb 28 18:10:59 2022", "message_details": "Prototype detection started ...."}
{"appname": "

In [ ]:
#print(results)

In [17]:
#Read output and print
import json
metrics = results.get("metrics_result")
#explain_metrics = metrics.get("explainability")
protodash_metrics = metrics.get("explainability").get("protodash")
        
if not protodash_metrics:
    print("Unable to detect prototypes for input data")
else:
    #print(protodash_metrics)
    pass

### Case-1: Display prototypes of the applicant classified as Risk

In [21]:
#Display prototypes with weights( No Risk)
print("=======input_df======")
display(test_df[199:200])
print("=======Prototypes======")
prototypes = protodash_metrics[0].get("prototypes")
prototypes_df = pd.DataFrame.from_records(prototypes.get("values"),columns=prototypes.get("fields"))
display(prototypes_df.T)

=======input_df======


,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,Housing_IX,Job_IX,Telephone_IX,ForeignWorker_IX,label,features,rawPrediction,probability,prediction,predictedLabel
199,0_to_200,25,outstanding_credit,appliances,5049,greater_1000,greater_7,3,male,none,...,0.0,0.0,0.0,0.0,0.0,"[2.0, 2.0, 4.0, 3.0, 2.0, 0.0, 0.0, 1.0, 0.0, ...","[8.138649061599631, 11.861350938400367]","[0.40693245307998155, 0.5930675469200184]",1.0,Risk


=======Prototypes======


,0,1,2,3,4
CheckingStatus,no_checking,0_to_200,0_to_200,0_to_200,no_checking
LoanDuration,26.0,6.0,35.0,34.0,40.0
CreditHistory,outstanding_credit,prior_payments_delayed,prior_payments_delayed,prior_payments_delayed,prior_payments_delayed
LoanPurpose,appliances,car_new,appliances,furniture,furniture
LoanAmount,5556.0,3452.0,7842.0,250.0,11347.0
ExistingSavings,greater_1000,less_100,less_100,500_to_1000,unknown
EmploymentDuration,greater_7,4_to_7,1_to_4,greater_7,4_to_7
InstallmentPercent,5.0,3.0,3.0,4.0,4.0
Sex,male,male,male,female,male
OthersOnLoan,none,co-applicant,co-applicant,none,co-applicant


In [22]:
#Display feature weights of prototypes
# Feature weights define the similarity of prototypes with original row 
print("=======Feature weights of protoypes======")
feature_weights = protodash_metrics[0].get("feature_weights")
feature_weights_df = pd.DataFrame.from_records(feature_weights.get("values"),columns=feature_weights.get("fields"))
display(feature_weights_df.T)

=======Feature weights of protoypes======


,0,1,2,3,4
CheckingStatus,0.13,1.00,1.00,1.00,0.13
LoanDuration,0.92,0.20,0.43,0.47,0.29
CreditHistory,1.00,0.08,0.08,0.08,0.08
LoanPurpose,1.00,0.41,1.00,0.11,0.11
LoanAmount,0.87,0.66,0.48,0.28,0.19
ExistingSavings,1.00,0.38,0.38,0.38,0.14
EmploymentDuration,1.00,0.26,0.07,1.00,0.26
InstallmentPercent,0.07,1.00,1.00,0.26,0.26
Sex,1.00,1.00,1.00,0.08,1.00
OthersOnLoan,1.00,0.13,0.13,1.00,0.13


### Explanation:
The above table depicts the five closest user profiles to the chosen applicant. Based on importance weight outputted by the method we see that the prototype under column zero is the most representative user profile by far. About 12 features(feature-weight=1) out of 23 of this prototype are highly similar  to that of the user we want to explain. Also the bank employee can see that the applicant belongs to a group of rejected applicants with similar deliquency behavior. Realizing that the user also poses similar risk as these other applicants whose loan was rejected, the employee takes the more conservative decision of rejecting the users application as well.

### Case-2: Display prototypes of the applicant classified as No Risk

In [23]:
#Display prototypes with weights( Risk)
print("=======input_df======")
display(test_df[200:201])
print("=======Prototypes======")
prototypes = protodash_metrics[1].get("prototypes")
prototypes_df = pd.DataFrame.from_records(prototypes.get("values"),columns=prototypes.get("fields"))
display(prototypes_df.T)

=======input_df======


,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,Housing_IX,Job_IX,Telephone_IX,ForeignWorker_IX,label,features,rawPrediction,probability,prediction,predictedLabel
0,0_to_200,4,all_credits_paid_back,car_new,250,less_100,1_to_4,1,female,none,...,0.0,3.0,0.0,0.0,0.0,"[2.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 2.0, ...","[18.44371785798895, 1.5562821420110489]","[0.9221858928994475, 0.07781410710055245]",0.0,No Risk


=======Prototypes======


,0,1,2,3,4
CheckingStatus,0_to_200,less_0,less_0,0_to_200,no_checking
LoanDuration,4.0,18.0,15.0,28.0,9.0
CreditHistory,all_credits_paid_back,credits_paid_to_date,prior_payments_delayed,credits_paid_to_date,prior_payments_delayed
LoanPurpose,car_new,car_new,furniture,retraining,car_new
LoanAmount,250.0,462.0,250.0,3693.0,1032.0
ExistingSavings,less_100,less_100,less_100,less_100,100_to_500
EmploymentDuration,1_to_4,1_to_4,1_to_4,greater_7,4_to_7
InstallmentPercent,1.0,2.0,2.0,3.0,3.0
Sex,female,female,male,male,male
OthersOnLoan,none,none,none,none,none


In [24]:
#Display feature weights of prototypes
# Feature weights define the similarity of prototypes with original row 
print("=======Feature weights of protoypes======")
feature_weights = protodash_metrics[1].get("feature_weights")
feature_weights_df = pd.DataFrame.from_records(feature_weights.get("values"),columns=feature_weights.get("fields"))
display(feature_weights_df.T)

=======Feature weights of protoypes======


,0,1,2,3,4
CheckingStatus,1.0,0.19,0.19,1.00,0.08
LoanDuration,1.0,0.18,0.26,0.05,0.54
CreditHistory,1.0,0.55,0.09,0.55,0.09
LoanPurpose,1.0,1.00,0.34,0.08,1.00
LoanAmount,1.0,0.85,1.00,0.07,0.55
ExistingSavings,1.0,1.00,1.00,1.00,0.08
EmploymentDuration,1.0,1.00,1.00,0.08,0.29
InstallmentPercent,1.0,0.26,0.26,0.07,0.07
Sex,1.0,1.00,0.13,0.13,0.13
OthersOnLoan,1.0,1.00,1.00,1.00,1.00


###  Shutdown the spark

In [ ]:
spark.stop()